In [1]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import warnings
from sklearn import metrics

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('ds_challenge_data_final.csv', sep='\t')
df.fillna('none', inplace = True)
df.fillna(0, inplace = True)
df.columns

Index(['Hash_Trans_ID', 'SKU_ID_Count', 'Transaction_Date', 'Transaction_Year',
       'Transaction_Month', 'Transaction_Day', 'Transaction_Hour',
       'Transaction_Day_of_Week', 'Transaction_Week',
       'Transaction_Day_of_Year',
       ...
       'Moving_Avg_Txn_Transaction_Type_Front line - Sales',
       'Moving_Avg_Txn_Transaction_Type_Front line -Self Chk',
       'Moving_Avg_Txn_Transaction_Type_Garden Sales',
       'Moving_Avg_Txn_Transaction_Type_Pro Desk - Refund',
       'Moving_Avg_Txn_Transaction_Type_Pro Desk - Sales',
       'Moving_Avg_Txn_Transaction_Type_Returns desk - Refnd',
       'Moving_Avg_Txn_Transaction_Type_Sp. Srvc. Dsk  VPOS',
       'Moving_Avg_Txn_Transaction_Type_Sp. Srvc. Dsk Refnd',
       'Moving_Avg_Txn_Transaction_Type_Tool Rental -Refund',
       'Moving_Avg_Txn_Transaction_Type_Tools - Refund'],
      dtype='object', length=152)

In [3]:
pred_cat_cols = ['holiday_nm']

In [4]:
cols_to_drop = ['Hash_Trans_ID','PRO', 'Transaction_Date']

In [5]:
df = pd.get_dummies(data=df, columns = pred_cat_cols)

In [6]:
df_train = df[df['Transaction_Year'] < 2015]
train_targets = df_train['PRO'].as_matrix().astype('float32')
train_data = df_train.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')

In [7]:
df_test = df[df['Transaction_Year'] == 2015]
test_targets = df_test['PRO'].as_matrix().astype('float32')
test_data = df_test.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')

In [8]:
mdl = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

In [9]:
mdl.fit(train_data,train_targets)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
test_pred = mdl.predict_proba(test_data)

In [14]:
predictions = np.array(test_pred)[:,1]
actuals = test_targets.astype('int')
fpr, tpr, thresholds = metrics.roc_curve(actuals, predictions, pos_label=1)
metrics.auc(fpr, tpr)

0.4990893641123752